In [1]:
"""
Three Layer MNIST
"""
def fetch(url):
    import gzip, os, urllib.request, hashlib, numpy as np
    filename = hashlib.md5(url.encode('utf-8')).hexdigest()
    if not os.path.isfile(filename): 
        dat = urllib.request.urlopen(url).read()
        with open(filename, 'wb') as f:
            f.write(dat)
    else: 
        with open(filename, 'rb') as f: 
            dat = f.read()
    return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

In [2]:
X_Train = fetch('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz')#[0x10:].reshape((-1, 28, 28))
Y_Train = fetch('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz')#[8:]
x_Test = fetch('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz')[0:784*2].reshape((-1, 28, 28))
y_Test = fetch('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz')[8:]


In [3]:
# shape the data into its respective inputs
import numpy as np
#from matplotlib.pyplot import imshow
#import matplotlib.pyplot as plt
import sys

images = np.reshape(X_Train[0x10:], (-1,28,28))[:1000]
images = images/255
labels = Y_Train[8:]


# images[0] -> notice how the pixelation of the images are in the shape of a number
# also note, the pixelation variations are from 0/255 values.

# run imshow(images[0]) to show the image

In [4]:
images = images.reshape(len(images), 28*28) # reshape again into a matrix of len(images) rows. 
# make the length of each row 28*28 in length
# this essentially flattens everything into a twodimensional matrix.




hot_labels = np.zeros((len(labels), 10)) # 1000, 10 matrix
for i, l in enumerate(labels): 
    hot_labels[i] = 1


test_images = x_Test.reshape(len(x_Test), 28*28)
test_labels = np.zeros((len(y_Test), 10))
for i, l in enumerate(y_Test):
    test_labels[i] = 1
    



In [5]:


np.random.seed(0)

relu = lambda x: (x>0) * x
relu2deriv = lambda x: (x>0)

alpha = 0.0005
iterations = 350
hidden_size = 40
pixels_per_image = 28*28
num_labels = 10 # all numbers ranging from 0-10



In [6]:
weights_0_1 = .2*np.random.random((pixels_per_image, hidden_size)) - .1
weights_1_2 = .2*np.random.random((hidden_size, num_labels)) - .1



In [7]:
#from tqdm import tqdm
#from tqdm import trange
#import matplotlib.pyplot as plt

for j in range(iterations):#tqdm(t:=range(iterations), desc=("Error: " + str(error/float(len(images)))[0:5] + " Prediction: " + str(correct_cnt/float(len(images))))):
    
    error, correct_cnt = (0.0, 0)

    for i in range(len(images)):
        
        # define layers of the network - tri layer. 
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        
        #addition of dropout mask. Multiply a small subset of matrix 1,0's to randomly switch off half the nodes
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        #apply the dropout mask to this layer
        layer_1 *= dropout_mask * 2
        
        layer_2 = np.dot(layer_1, weights_1_2)
        
        # calculate error values
        error += np.sum((layer_2 - labels[i:i+1]) ** 2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))

        # find delta values
        delta_2 = (layer_2 - labels[i:i+1])
        delta_1 = delta_2.dot(weights_1_2.T)
        
        delta_1 *= dropout_mask
        
        # apply activation function / apply non-linearity
        d_delta_1 = delta_1*relu2deriv(layer_1)
        
        # update the weights
        weights_1_2 -= (alpha * layer_1.T.dot(delta_2))
        weights_0_1 -= (alpha * layer_0.T.dot(d_delta_1))
        
        if (j%10 == 0): 
            print(j)
            test_error = 0.0
            test_correct_cnt = 0
            
            for i in range(len(test_images)):
                layer_0 = test_images[i:i+1]
                layer_1 = relu(np.dot(layer_0, weights_0_1))
                layer_2 = np.dot(layer_1, weights_1_2)
                
                test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
                test_correct_cnt += int(np.argmax(layer_2) == \
                                       np.argmax(test_labels[i:i+1]))
            print(True)
        
            sys.stdout.write("\n" + \
                            "I: " + str(j) + \
                            " Test-Err: " + str(test_error/ float(len(test_images))) [0:5] + \
                            " Train-Err: " + str(error/ float(len(images))) [0:5] + \
                            " Train-Acc: " + str(correct_cnt/float(len(images))))
#plt.plot(weights)
#plt.plot(labels)
#plt.show()


    

0
True

I: 0 Test-Err: 11004 Train-Err: 0.245 Train-Acc: 0.00
True

I: 0 Test-Err: 11004 Train-Err: 0.245 Train-Acc: 0.00
True

I: 0 Test-Err: 10915 Train-Err: 0.406 Train-Acc: 0.0010
True

I: 0 Test-Err: 10915 Train-Err: 0.416 Train-Acc: 0.0010
True

I: 0 Test-Err: 11012 Train-Err: 1.213 Train-Acc: 0.0010
True

I: 0 Test-Err: 11077 Train-Err: 1.250 Train-Acc: 0.0010
True

I: 0 Test-Err: 11085 Train-Err: 1.259 Train-Acc: 0.0010
True

I: 0 Test-Err: 11243 Train-Err: 1.340 Train-Acc: 0.0010
True

I: 0 Test-Err: 11247 Train-Err: 1.349 Train-Acc: 0.0010
True

I: 0 Test-Err: 11327 Train-Err: 1.504 Train-Acc: 0.0010
True

I: 0 Test-Err: 11379 Train-Err: 1.585 Train-Acc: 0.0010
True

I: 0 Test-Err: 11530 Train-Err: 1.828 Train-Acc: 0.0010
True

I: 0 Test-Err: 11792 Train-Err: 1.912 Train-Acc: 0.0020
True

I: 0 Test-Err: 12210 Train-Err: 2.266 Train-Acc: 0.0030
True

I: 0 Test-Err: 12214 Train-Err: 2.275 Train-Acc: 0.0030
True

I: 0 Test-Err: 12698 Train-Err: 2.742 Train-Acc: 0.0030
True

I: 0

I: 0 Test-Err: 24789 Train-Err: 34.19 Train-Acc: 0.0470
True

I: 0 Test-Err: 23911 Train-Err: 34.22 Train-Acc: 0.0470
True

I: 0 Test-Err: 23890 Train-Err: 34.22 Train-Acc: 0.0470
True

I: 0 Test-Err: 22548 Train-Err: 34.25 Train-Acc: 0.0470
True

I: 0 Test-Err: 23162 Train-Err: 34.34 Train-Acc: 0.0470
True

I: 0 Test-Err: 22553 Train-Err: 34.35 Train-Acc: 0.0480
True

I: 0 Test-Err: 23323 Train-Err: 34.57 Train-Acc: 0.0480
True

I: 0 Test-Err: 24087 Train-Err: 34.62 Train-Acc: 0.0480
True

I: 0 Test-Err: 25072 Train-Err: 35.23 Train-Acc: 0.0480
True

I: 0 Test-Err: 23676 Train-Err: 35.27 Train-Acc: 0.0480
True

I: 0 Test-Err: 24091 Train-Err: 35.44 Train-Acc: 0.0480
True

I: 0 Test-Err: 23861 Train-Err: 35.45 Train-Acc: 0.0480
True

I: 0 Test-Err: 21384 Train-Err: 35.62 Train-Acc: 0.0480
True

I: 0 Test-Err: 21680 Train-Err: 35.64 Train-Acc: 0.0480
True

I: 0 Test-Err: 22259 Train-Err: 35.78 Train-Acc: 0.0480
True

I: 0 Test-Err: 21413 Train-Err: 35.84 Train-Acc: 0.0480
True

I: 0 Tes

I: 0 Test-Err: 17727 Train-Err: 54.10 Train-Acc: 0.0790
True

I: 0 Test-Err: 17850 Train-Err: 54.10 Train-Acc: 0.0790
True

I: 0 Test-Err: 17846 Train-Err: 54.10 Train-Acc: 0.0790
True

I: 0 Test-Err: 17564 Train-Err: 54.16 Train-Acc: 0.0790
True

I: 0 Test-Err: 17511 Train-Err: 54.16 Train-Acc: 0.0790
True

I: 0 Test-Err: 17501 Train-Err: 54.16 Train-Acc: 0.0790
True

I: 0 Test-Err: 19582 Train-Err: 54.28 Train-Acc: 0.0790
True

I: 0 Test-Err: 20328 Train-Err: 54.38 Train-Acc: 0.0790
True

I: 0 Test-Err: 19296 Train-Err: 54.41 Train-Acc: 0.0790
True

I: 0 Test-Err: 20682 Train-Err: 54.47 Train-Acc: 0.0790
True

I: 0 Test-Err: 20751 Train-Err: 54.47 Train-Acc: 0.0790
True

I: 0 Test-Err: 20040 Train-Err: 54.51 Train-Acc: 0.0790
True

I: 0 Test-Err: 20421 Train-Err: 54.52 Train-Acc: 0.0790
True

I: 0 Test-Err: 21018 Train-Err: 54.56 Train-Acc: 0.080
True

I: 0 Test-Err: 21038 Train-Err: 54.56 Train-Acc: 0.080
True

I: 0 Test-Err: 20761 Train-Err: 54.57 Train-Acc: 0.080
True

I: 0 Test-E

True

I: 0 Test-Err: 31493 Train-Err: 72.93 Train-Acc: 0.0980
True

I: 0 Test-Err: 31995 Train-Err: 72.95 Train-Acc: 0.0980
True

I: 0 Test-Err: 30576 Train-Err: 73.11 Train-Acc: 0.0980
True

I: 0 Test-Err: 30127 Train-Err: 73.12 Train-Acc: 0.0980
True

I: 0 Test-Err: 30244 Train-Err: 73.12 Train-Acc: 0.0980
True

I: 0 Test-Err: 30289 Train-Err: 73.12 Train-Acc: 0.0980
True

I: 0 Test-Err: 30131 Train-Err: 73.13 Train-Acc: 0.0980
True

I: 0 Test-Err: 30305 Train-Err: 73.13 Train-Acc: 0.0980
True

I: 0 Test-Err: 30567 Train-Err: 73.14 Train-Acc: 0.0980
True

I: 0 Test-Err: 28608 Train-Err: 73.21 Train-Acc: 0.0980
True

I: 0 Test-Err: 29739 Train-Err: 73.28 Train-Acc: 0.0980
True

I: 0 Test-Err: 28929 Train-Err: 73.28 Train-Acc: 0.0990
True

I: 0 Test-Err: 29384 Train-Err: 73.29 Train-Acc: 0.10
True

I: 0 Test-Err: 28762 Train-Err: 73.33 Train-Acc: 0.10
True

I: 0 Test-Err: 29090 Train-Err: 73.37 Train-Acc: 0.10
True

I: 0 Test-Err: 29568 Train-Err: 73.38 Train-Acc: 0.1010
True

I: 0 Tes

I: 0 Test-Err: 25793 Train-Err: 83.11 Train-Acc: 0.11810
True

I: 10 Test-Err: 90999 Train-Err: 0.006 Train-Acc: 0.010
True

I: 10 Test-Err: 90533 Train-Err: 0.013 Train-Acc: 0.010
True

I: 10 Test-Err: 92196 Train-Err: 0.071 Train-Acc: 0.010
True

I: 10 Test-Err: 92229 Train-Err: 0.072 Train-Acc: 0.010
True

I: 10 Test-Err: 92132 Train-Err: 0.072 Train-Acc: 0.010
True

I: 10 Test-Err: 91805 Train-Err: 0.076 Train-Acc: 0.010
True

I: 10 Test-Err: 91416 Train-Err: 0.085 Train-Acc: 0.010
True

I: 10 Test-Err: 91206 Train-Err: 0.086 Train-Acc: 0.010
True

I: 10 Test-Err: 90856 Train-Err: 0.100 Train-Acc: 0.00110
True

I: 10 Test-Err: 90857 Train-Err: 0.100 Train-Acc: 0.00110
True

I: 10 Test-Err: 88481 Train-Err: 0.154 Train-Acc: 0.00110
True

I: 10 Test-Err: 92121 Train-Err: 0.244 Train-Acc: 0.00110
True

I: 10 Test-Err: 92338 Train-Err: 0.258 Train-Acc: 0.00110
True

I: 10 Test-Err: 93535 Train-Err: 0.281 Train-Acc: 0.00110
True

I: 10 Test-Err: 93484 Train-Err: 0.281 Train-Acc: 0.00110

...

I: 330 Test-Err: 21607 Train-Err: 12.48 Train-Acc: 0.002330
True

I: 330 Test-Err: 21296 Train-Err: 12.49 Train-Acc: 0.002330
True

I: 330 Test-Err: 21466 Train-Err: 12.50 Train-Acc: 0.002330
True

I: 330 Test-Err: 21451 Train-Err: 12.50 Train-Acc: 0.002330
True

I: 330 Test-Err: 21153 Train-Err: 12.51 Train-Acc: 0.002330
True

I: 330 Test-Err: 21787 Train-Err: 12.57 Train-Acc: 0.002330
True

I: 330 Test-Err: 21566 Train-Err: 12.58 Train-Acc: 0.002330
True

I: 330 Test-Err: 21411 Train-Err: 12.59 Train-Acc: 0.002330
True

I: 330 Test-Err: 21539 Train-Err: 12.60 Train-Acc: 0.002330
True

I: 330 Test-Err: 21512 Train-Err: 12.60 Train-Acc: 0.002330
True

I: 330 Test-Err: 21452 Train-Err: 12.60 Train-Acc: 0.002330
True

I: 330 Test-Err: 21837 Train-Err: 12.65 Train-Acc: 0.002330
True

I: 330 Test-Err: 21995 Train-Err: 12.65 Train-Acc: 0.002330
True

I: 330 Test-Err: 22065 Train-Err: 12.67 Train-Acc: 0.002330
True

I: 330 Test-Err: 21994 Train-Err: 12.67 Train-Acc: 0.002330
True

I: 330 Tes

I: 330 Test-Err: 24854 Train-Err: 15.29 Train-Acc: 0.002340
True

I: 340 Test-Err: 14701 Train-Err: 0.033 Train-Acc: 0.0340
True

I: 340 Test-Err: 14702 Train-Err: 0.033 Train-Acc: 0.0340
True

I: 340 Test-Err: 14669 Train-Err: 0.033 Train-Acc: 0.0340
True

I: 340 Test-Err: 14679 Train-Err: 0.033 Train-Acc: 0.0340
True

I: 340 Test-Err: 15358 Train-Err: 0.111 Train-Acc: 0.0340
True

I: 340 Test-Err: 15333 Train-Err: 0.111 Train-Acc: 0.0340
True

I: 340 Test-Err: 15337 Train-Err: 0.119 Train-Acc: 0.0340
True

I: 340 Test-Err: 15367 Train-Err: 0.119 Train-Acc: 0.0340
True

I: 340 Test-Err: 15347 Train-Err: 0.123 Train-Acc: 0.0340
True

I: 340 Test-Err: 15577 Train-Err: 0.141 Train-Acc: 0.0340
True

I: 340 Test-Err: 15577 Train-Err: 0.141 Train-Acc: 0.0340
True

I: 340 Test-Err: 15841 Train-Err: 0.155 Train-Acc: 0.0340
True

I: 340 Test-Err: 16010 Train-Err: 0.164 Train-Acc: 0.0340
True

I: 340 Test-Err: 16096 Train-Err: 0.165 Train-Acc: 0.0340
True

I: 340 Test-Err: 16085 Train-Err: 0.16

True

I: 340 Test-Err: 16469 Train-Err: 4.232 Train-Acc: 0.002340
True

I: 340 Test-Err: 16660 Train-Err: 4.241 Train-Acc: 0.002340
True

I: 340 Test-Err: 16744 Train-Err: 4.245 Train-Acc: 0.002340
True

I: 340 Test-Err: 16845 Train-Err: 4.246 Train-Acc: 0.002340
True

I: 340 Test-Err: 16857 Train-Err: 4.246 Train-Acc: 0.002340
True

I: 340 Test-Err: 16455 Train-Err: 4.255 Train-Acc: 0.002340
True

I: 340 Test-Err: 16447 Train-Err: 4.255 Train-Acc: 0.002340
True

I: 340 Test-Err: 16442 Train-Err: 4.255 Train-Acc: 0.002340
True

I: 340 Test-Err: 16040 Train-Err: 4.270 Train-Acc: 0.002340
True

I: 340 Test-Err: 15663 Train-Err: 4.286 Train-Acc: 0.002340
True

I: 340 Test-Err: 15602 Train-Err: 4.286 Train-Acc: 0.002340
True

I: 340 Test-Err: 15883 Train-Err: 4.302 Train-Acc: 0.002340
True

I: 340 Test-Err: 15713 Train-Err: 4.311 Train-Acc: 0.002340
True

I: 340 Test-Err: 15713 Train-Err: 4.311 Train-Acc: 0.002340
True

I: 340 Test-Err: 15836 Train-Err: 4.329 Train-Acc: 0.002340
True

I: 3

True

I: 340 Test-Err: 15465 Train-Err: 8.199 Train-Acc: 0.002340
True

I: 340 Test-Err: 15305 Train-Err: 8.205 Train-Acc: 0.002340
True

I: 340 Test-Err: 15455 Train-Err: 8.216 Train-Acc: 0.002340
True

I: 340 Test-Err: 16230 Train-Err: 8.381 Train-Acc: 0.002340
True

I: 340 Test-Err: 16246 Train-Err: 8.382 Train-Acc: 0.002340
True

I: 340 Test-Err: 16273 Train-Err: 8.382 Train-Acc: 0.002340
True

I: 340 Test-Err: 16495 Train-Err: 8.437 Train-Acc: 0.002340
True

I: 340 Test-Err: 16579 Train-Err: 8.441 Train-Acc: 0.002340
True

I: 340 Test-Err: 16410 Train-Err: 8.446 Train-Acc: 0.002340
True

I: 340 Test-Err: 16280 Train-Err: 8.451 Train-Acc: 0.002340
True

I: 340 Test-Err: 16276 Train-Err: 8.451 Train-Acc: 0.002340
True

I: 340 Test-Err: 16041 Train-Err: 8.461 Train-Acc: 0.002340
True

I: 340 Test-Err: 16440 Train-Err: 8.484 Train-Acc: 0.002340
True

I: 340 Test-Err: 16434 Train-Err: 8.484 Train-Acc: 0.002340
True

I: 340 Test-Err: 16476 Train-Err: 8.487 Train-Acc: 0.002340
True

I: 3

I: 340 Test-Err: 14642 Train-Err: 13.22 Train-Acc: 0.002340
True

I: 340 Test-Err: 15045 Train-Err: 13.26 Train-Acc: 0.002340
True

I: 340 Test-Err: 14769 Train-Err: 13.27 Train-Acc: 0.002340
True

I: 340 Test-Err: 15057 Train-Err: 13.30 Train-Acc: 0.002340
True

I: 340 Test-Err: 14988 Train-Err: 13.31 Train-Acc: 0.002340
True

I: 340 Test-Err: 14626 Train-Err: 13.33 Train-Acc: 0.002340
True

I: 340 Test-Err: 14647 Train-Err: 13.34 Train-Acc: 0.002340
True

I: 340 Test-Err: 14948 Train-Err: 13.37 Train-Acc: 0.002340
True

I: 340 Test-Err: 15320 Train-Err: 13.41 Train-Acc: 0.002340
True

I: 340 Test-Err: 15216 Train-Err: 13.41 Train-Acc: 0.002340
True

I: 340 Test-Err: 15272 Train-Err: 13.41 Train-Acc: 0.002340
True

I: 340 Test-Err: 15261 Train-Err: 13.41 Train-Acc: 0.002340
True

I: 340 Test-Err: 15350 Train-Err: 13.41 Train-Acc: 0.002340
True

I: 340 Test-Err: 15533 Train-Err: 13.42 Train-Acc: 0.002340
True

I: 340 Test-Err: 14899 Train-Err: 13.46 Train-Acc: 0.002340
True

I: 340 Tes